In [1]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
import time
import re
from pprint import pprint
import glob
import os
import warnings
warnings.filterwarnings('ignore')


# Read Player Database

In [2]:
players_df = pd.read_csv('Player Database - Singapore.csv',
                        names=('ID','First Name','Last Name'),usecols=[0,1,2])

In [7]:
csv_dir = "C:\\misc\\scba-techcomm\\masterpoints\\csv_2019\\"

# Helper Functions

## Player Number Lookup

In [3]:
def find_player_numbers(name, table_df):
    name_words = name.split(' ')
    last_name = name_words[-1]
    first_name = ' '.join(name_words[:-1])
    
    try:
        player_id = int(players_df[(players_df['First Name']==first_name) & (players_df['Last Name']==last_name)]['ID'].item())
        return player_id
    except:
        return None

In [89]:
def ext_meta(df_meta):
    event_meta_df = pd.DataFrame()
    event_meta_df['Date'] = df_meta.loc['#Date']
    event_meta_df['Event'] = df_meta.loc['#Title']
    event_meta_df['Event_ID'] = df_meta.loc['Event Number']
    event_meta_df['Director'] = df_meta.loc['#Director']
    event_meta_df['Boards'] = int(df_meta.loc['#Boards'].value)
    try:
        event_meta_df['Winners'] = int(df_meta.loc['#Winners'].value)
    except:
        event_meta_df['Winners'] = 1
    try:
        event_meta_df['Tables'] = int(df_meta.loc['#Tables'].value)
    except:
        event_meta_df['Tables'] = 0
    try:
        event_meta_df['Rounds'] = int(df_meta.loc['#Rounds'].value)
    except:
        event_meta_df['Rounds'] = 0
    try:
        event_meta_df['Matches'] = int(df_meta.loc['#Matches'].value)
    except:
        event_meta_df['Matches'] = 0
    try:
        event_meta_df['Phantom'] = int(df_meta.loc['#Phantom'].value)
    except:
        event_meta_df['Phantom'] = 0
    
    event_meta_df.reset_index()
    
    return event_meta_df

# Read all metadata

In [90]:
all_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"*.csv"):
    print(csv_file)
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20,)
    event_meta_df = ext_meta(df_meta)
    
    filename = csv_file.split('\\')[-1]
    event_meta_df['Filename'] = filename
    event_meta_df['Source'] = filename[0:2]
    
    all_event_meta_df = all_event_meta_df.append(event_meta_df,ignore_index=True)

all_event_meta_df['Date'] = pd.to_datetime(all_event_meta_df['Date'],dayfirst=True)

C:\misc\scba-techcomm\masterpoints\csv_2019\ISNov23-2019-2.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr01-2019-244.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr04-2019-245.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr05-2019-246.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr08-2019-247.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr11-2019-248.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr12-2019-249.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr15-2019-250.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr18-2019-1.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-14.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-2.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-5.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr22-2019-6.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr25-2019-7.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr26-2019-8.csv
C:\misc\scba-techcomm\masterpoints\csv_2

C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov01-2019-98.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov02-2019-101.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov04-2019-102.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov07-2019-103.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov08-2019-73.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov11-2019-74.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov14-2019-76.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov15-2019-77.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov18-2019-78.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov21-2019-79.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov22-2019-80.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov25-2019-81.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov28-2019-82.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov29-2019-83.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct03-2019-69.csv
C:\misc\scba-techcomm\masterpoints\cs

In [91]:
all_event_meta_df

,Date,Event,Event_ID,Director,Boards,Winners,Tables,Rounds,Matches,Phantom,Filename,Source
0,2019-11-23,SCBA Individual Game 2019,2,Shen Ting Ang,24,1,2,8,0,0,ISNov23-2019-2.csv,IS
1,2019-04-01,Monday Pairs,244,Greta Chai,24,1,11,12,0,0,PSApr01-2019-244.csv,PS
2,2019-04-04,Thursday Pairs,245,Greta Chai,24,1,7,12,0,0,PSApr04-2019-245.csv,PS
3,2019-04-05,Friday Pairs,246,Peter Haw,24,1,9,12,0,0,PSApr05-2019-246.csv,PS
4,2019-04-08,Monday Pairs,247,Tan Wen Yuan,24,1,8,12,0,0,PSApr08-2019-247.csv,PS
5,2019-04-11,Thursday Pairs,248,Greta Chai,24,1,10,12,0,5,PSApr11-2019-248.csv,PS
6,2019-04-12,Friday Pairs,249,Tan Wen Yuan,24,2,12,12,0,0,PSApr12-2019-249.csv,PS
7,2019-04-15,Monday Pairs,250,Greta Chai,24,1,7,12,0,12,PSApr15-2019-250.csv,PS
8,2019-04-18,Thursday Pairs,1,Greta Chai,24,1,9,12,0,0,PSApr18-2019-1.csv,PS
9,2019-04-19,Easter Congress IMP Pairs,14,Wei Kang Gan,28,1,12,14,0,24,PSApr19-2019-14.csv,PS


In [92]:
all_event_meta_df.sort_values('Date',inplace=True)

In [93]:
all_event_meta_df.to_csv('all_meta_2019.csv', index=False)

We need to label events manually after this step.

# Read Metadata file in again after labelling

In [79]:
def calc_rank_mp(in_df, boards, tables, scale='Club', qualifinal=False, single_winner=True):
    """
    calc_mp: calculates masterpoints for ranks
    Inputs -
    in_df: DataFrame of results (which does not have Masterpoints column)
    boards: Number of boards played
    tables: Number of FULL tables
    scale: 'Club', 'District', 'County', 'Regional', 'National' (default = 'Club')
    single_winner: True if 1-winner pairs, False if 2-winner pairs (default=True)
    Returns -
    out_df: DataFrame of only masterpoint winners with 'MPTS' column appended
    
    Reference: Masterpoint Handbook Scale A-E
    """
    if scale=='National':
        scale_factor = 4
    elif scale =='Regional':
        scale_factor = 3
    elif scale =='County':
        scale_factor = 2
    elif scale =='District':
        scale_factor = 1.5
    elif scale == 'Club':
        scale_factor = 1
    else:
        scale_factor = 1
    
    
    if single_winner:
        base_factor = 6*scale_factor
        if boards < 12:
            n_awards = 0
            return None
        elif boards >= 12 and boards < 18:
            factor = 2
            if tables >=3:
                n_awards = math.ceil(tables/factor)
            else:
                n_awards = 0
                return None
        elif boards >= 18 and boards < 36:
            factor = 1.5
            if tables >=3:
                n_awards = math.ceil(tables/factor)
            else:
                n_awards = 0
                return None
        else:
            n_awards = tables # Factor is 1
        
        out_df = in_df[in_df['Position']<=n_awards]
        out_df['MPTS'] = base_factor*(n_awards+1-out_df['Position'])
        return out_df
        
    else:
        base_factor = 10*scale_factor
        if boards < 12:
            return None
        elif boards >= 12 and boards < 18:
            factor = 4
            # Limit at 30 tables
            if tables > 30:
                tables = 30
            if tables >=3:
                n_awards = round(tables/factor, 0)
            else:
                return None
        elif boards >= 18 and boards < 36:
            factor = 3
            # Limit at 30 tables
            if tables > 30:
                tables = 30
            if tables >=3:
                n_awards = round(tables/factor, 0)
            else:
                return None
        else:
            factor = 2
            if tables >=2:
                n_awards = round(tables/factor, 0)
        out_df = in_df[in_df['Position']<=n_awards]
        out_df['MPTS'] = base_factor*(n_awards+1-out_df['Position'])
        out_df['Award_Type'] = 'Ranking'
        return out_df

In [ ]:
def calc_match_mp(df, boards_per_round, scale='Club', ko=False, pairs=False, rd=0):
    """
    calc_match_mp: calculates masterpoints for matches
    Inputs -
    in_df: DataFrame of results 
    boards_per_round: Number of boards played per round
    scale: 'Club', 'District', 'County', 'Regional', 'National' (default = 'Club')
    ko: If matches are knockouts
    Returns -
    out_df: DataFrame of only masterpoint winners with 'MPTS' column appended
    
    Reference: Masterpoint Handbook Scale A-E
    """
    if not ko:  # Normal RR Matches  
        if scale=='National':
            scale_factor = 6
        elif scale =='Regional':
            scale_factor = 4
        elif scale =='County':
            scale_factor = 3
        elif scale =='District':
            scale_factor = 2.25
        elif scale == 'Club':
            scale_factor = 1.5
        else:
            scale_factor = 1.5
        
        if boards_per_round <18:
            boards = boards_per_round
        elif boards_per_round >= 18 and boards_per_round <36:
            boards = 18
        else:
            boards = 27
            
        mp_per_win = math.ceil(boards*scale_factor)
        
        df['Wins'] = # Filter
        df['MPTS'] = df['Wins']*mp_per_win
        df['Award_Type'] = 'Matches'
        
        return df
    # End Non-KO Matches
    else:
        if scale=='National':
            scale_factor = 4
        elif scale =='Regional':
            scale_factor = 3
        elif scale =='County':
            scale_factor = 2
        elif scale =='District':
            scale_factor = 1.5
        elif scale == 'Club':
            scale_factor = 1
        else:
            scale_factor = 1
            
        if boards_per_round <18:
            board_factor = boards_per_round/18
        elif boards_per_round >= 18 and boards_per_round <36:
            board_factor = 1
        else:
            boards_factor = 1
            rd += 1
        
        if pairs:
            rd -= 1
            
        if rd <= 1:
            base = 27
        elif rd == 2:
            base = 38
        elif rd == 3:
            base = 54
        elif rd == 4:
            base = 76
        elif rd == 5:
            base = 108
        elif rd >= 6:
            base = 152
        else:
            base = 27
        
        mp_per_win = math.ceil(base*board_factor*scale_factor)
        df['Wins'] = # Filter
        df['MPTS'] = df['Wins']*mp_per_win
        df['Award_Type'] = 'Matches'
        
        

In [84]:
def ext_pairs_winning_names(df, boards, tables, scale='Club', single_winner=True):
    df_winners = calc_rank_mp(df, boards, tables, scale, single_winner)
    
    if df_winners is not None:
        df_winners1 = df_winners.drop('Name2', axis=1)
        df_winners2 = df_winners.drop('Name1', axis=1)
    
        df_winners1.rename(columns={'Name1': 'Name'},inplace=True)
        df_winners2.rename(columns={'Name2': 'Name'},inplace=True)
    
        out_df = df_winners1.append(df_winners2)
        return out_df
    else:
        return None

In [94]:
def calc_n_pairs(tables, phantom):
    if phantom > 0:
        n_phantom = 1
    else:
        n_phantom = 0
                
    return tables*2 - n_phantom

In [95]:
def read_pairs_results(csv_file, n_pairs):
    # Read actual results
    df = pd.read_csv(csv_file, sep=';',comment='#',header=1, nrows=n_pairs+1, error_bad_lines=False,warn_bad_lines=False)
    if 'Position' in df['Position'].values:
        df.drop(df[df['Position']=='Position'].index, inplace=True) # Clear up extra header for 2-winner
    pairs_df = df.iloc[0:int(n_pairs)].sort_values('Position',ascending=True)
    
    if 'NS' in pairs_df['Position'].values:
        pairs_df.drop(pairs_df[pairs_df['Position']=='NS'].index, inplace=True) # Clear up extra header for missing phantom
        n_tables = n_tables - 1
        
    pairs_df['Position'] = pairs_df['Position'].astype('int') # Convert position to int  
    
    return pairs_df

## Read in meta

In [104]:
all_event_meta_df = pd.read_csv('all_meta.csv')

In [105]:
all_event_meta_df.head()

,Date,Actual_Event,Event,Event_ID,Director,Winners,Boards,Tables,Rounds,Matches,Phantom,Filename,Source,Scale,NLP_Eligible,Qualifying,Final,Consols,KO
0,2019-01-01,New Year Pairs,Tuesday Pairs New Year Day,200.0,Greta Chai,1,24.0,7.0,12.0,0.0,12.0,PSJan01-2019-200.csv,PS,District,False,False,False,False,False
1,2019-01-03,Thursday Pairs,Thursday Pairs,201.0,Greta Chai,1,24.0,8.0,12.0,0.0,0.0,PSJan03-2019-201.csv,PS,Club,False,False,False,False,False
2,2019-01-04,Friday Pairs,Friday Pairs,202.0,Siew Kheng Hun,1,24.0,11.0,12.0,0.0,0.0,PSJan04-2019-202.csv,PS,Club,False,False,False,False,False
3,2019-01-05,Mixed Team Trials,Mixed Trials 2019,120.0,Gerald Tan,1,84.0,5.0,0.0,5.0,0.0,TSJan05-2019-120.csv,TS,County,False,True,False,False,False
4,2019-01-06,Mixed Team Trials,Mixed Team Trials Day 2,123.0,Gerald Tan,1,84.0,5.0,0.0,5.0,0.0,TSJan06-2019-123.csv,TS,County,False,True,False,False,False


In [106]:
unique_events = all_event_meta_df['Actual_Event'].unique() # Get unique events

### Test code - delete later

In [107]:
# Testing subset
all_event_meta_df = all_event_meta_df.iloc[0:15]
unique_events = all_event_meta_df['Actual_Event'].unique()

In [111]:
tmp_df = all_event_meta_df[all_event_meta_df['Actual_Event']=='NUS Open Pairs']

In [112]:
tmp_df

,Date,Actual_Event,Event,Event_ID,Director,Winners,Boards,Tables,Rounds,Matches,Phantom,Filename,Source,Scale,NLP_Eligible,Qualifying,Final,Consols,KO
8,2019-01-12,NUS Open Pairs,NUS Open Pairs,1.0,,1,32.0,16.0,14.0,0.0,0.0,PSJan12-2019-1.csv,PS,District,False,False,False,False,False


# Main Loop

In [116]:
def fix_teams_header(csv_file, n_teams):
    header = pd.read_csv(csv_file, sep=';',comment='#',header=1, nrows=0, error_bad_lines=False,warn_bad_lines=False)
    data = pd.read_csv(csv_file, sep=';',comment='#',header=2, nrows=int(n_teams)-2, error_bad_lines=False,warn_bad_lines=False)
    
    diff_cols = len(data.columns) - len(header.columns)
    header_list = list(header.columns)
    idx = header_list.index('Matches')
    for i in range(0,diff_cols):
        count = str(i+2)
        idx +=1
        header_list.insert(idx,'Matches'+count)
    
    return header_list

In [126]:
def ext_teams_winning_names(df, boards, tables, scale='Club', qualifinal=False, single_winner=False):
    df_winners = calc_rank_mp(in_df, boards, tables, scale, qualifinal, single_winner)
    
    if df_winners is not None:
        df_winners1 = df_winners.drop(['Name2','Name3','Name4','Name5','Name6','Name7','Name8'], axis=1)
        df_winners2 = df_winners.drop(['Name1','Name3','Name4','Name5','Name6','Name7','Name8'], axis=1)
        df_winners3 = df_winners.drop(['Name1','Name2','Name4','Name5','Name6','Name7','Name8'], axis=1)
        df_winners4 = df_winners.drop(['Name1','Name2','Name3','Name5','Name6','Name7','Name8'], axis=1)
        df_winners5 = df_winners.drop(['Name1','Name2','Name3','Name4','Name6','Name7','Name8'], axis=1)
        df_winners6 = df_winners.drop(['Name1','Name2','Name3','Name4','Name5','Name7','Name8'], axis=1)
        df_winners7 = df_winners.drop(['Name1','Name2','Name3','Name4','Name5','Name6','Name8'], axis=1)
        df_winners8 = df_winners.drop(['Name1','Name2','Name3','Name4','Name5','Name6','Name7'], axis=1)
    
        df_winners1.rename(columns={'Name1': 'Name'},inplace=True)
        df_winners2.rename(columns={'Name2': 'Name'},inplace=True)
        df_winners3.rename(columns={'Name3': 'Name'},inplace=True)
        df_winners4.rename(columns={'Name4': 'Name'},inplace=True)
        df_winners5.rename(columns={'Name5': 'Name'},inplace=True)
        df_winners6.rename(columns={'Name6': 'Name'},inplace=True)
        df_winners7.rename(columns={'Name7': 'Name'},inplace=True)
        df_winners8.rename(columns={'Name8': 'Name'},inplace=True)
    
        out_df = pd.concat([df_winners1, df_winners2, df_winners3, df_winners4, 
                            df_winners5, df_winners6, df_winners7, df_winners8])
        return out_df
    else:
        return None

In [118]:
def read_teams_results(csv_file, n_teams):
    
    # Read actual results
    header_list = fix_teams_header(csv_file, n_teams)
    df = pd.read_csv(csv_file, sep=';',comment='#',header=1, names=header_list, nrows=int(n_teams), error_bad_lines=False,warn_bad_lines=False)
    if 'Position' in df['Position'].values:
        df.drop(df[df['Position']=='Position'].index, inplace=True) # Clear up extra header for 2-winner
    teams_df = df.iloc[0:int(n_teams)].sort_values('Position',ascending=True)
    
    if 'NS' in teams_df['Position'].values:
        teams_df.drop(teams_df[teams_df['Position']=='NS'].index, inplace=True) # Clear up extra header for missing phantom
        n_tables = n_tables - 1
        
    teams_df['Position'] = teams_df['Position'].astype('int') # Convert position to int
    
    return teams_df

In [124]:
def recalc_scale(scale, final, consols):
    if scale == 'Club':
        if final and not consols:
            return 'District'
        elif consols and not final:
            return 'Club'
        else:
            return scale
    elif scale == 'District':
        if final and not consols:
            return 'County'
        elif consols and not final:
            return 'Club'
        else:
            return scale
    elif scale == 'County':
        if final and not consols:
            return 'Regional'
        elif consols and not final:
            return 'District'
        else:
            return scale
    elif scale == 'Regional':
        if final and not consols:
            return 'National'
        elif consols and not final:
            return 'County'
        else:
            return scale
    elif scale == 'National':
        if final and not consols:
            return 'National'
        elif consols and not final:
            return 'Regional'
        else:
            return scale
    else:
        return scale

In [119]:
all_mp_df = pd.DataFrame()
# Loop through
for event in unique_events:
    # Weekly games are not uniquely named, we need to handle them differently.
    
    if event in ['Monday Pairs', 'Thursday Pairs', 'Friday Pairs']:
        event_df = all_event_meta_df[all_event_meta_df['Actual_Event']==event]
        for row in event_df.itertuples():
            csv_file = csv_dir + row.Filename
            
            if int(row.Winners)!=1:
                single_winner=False
            else:
                single_winner=True
                
            n_pairs = calc_n_pairs(row.Tables, row.Phantom)
            
            pairs_df = read_pairs_results(csv_file, n_pairs)
        
            out_df = ext_pairs_winning_names(pairs_df, row.Boards, row.Tables, row.Scale, single_winner)
    
            if out_df is not None:
    
                out_df['Player ID'] = out_df['Name'].apply(find_player_numbers, args = (players_df,))
                out_df['Date'] = row.Date
                out_df['Event'] = row.Event
                out_df['Event_ID'] = row.Event_ID
                
            all_mp_df = all_mp_df.append(out_df)
    # End Weekly Games
    else:
        event_df = all_event_meta_df[all_event_meta_df['Actual_Event']==event]
        if len(event_df)==1:
            # Single Event, just score directly
            for row in event_df.itertuples():
                csv_file = csv_dir + row.Filename
                if row.Source == 'PS': # Pairs
                    if int(row.Winners)!=1:
                        single_winner=False
                    else:
                        single_winner=True
                
                    n_pairs = calc_n_pairs(row.Tables, row.Phantom)
            
                    pairs_df = read_pairs_results(csv_file, n_pairs)
        
                    out_df = ext_pairs_winning_names(pairs_df, row.Boards, row.Tables, row.Scale, single_winner)
    
                    if out_df is not None:
                        out_df['Player ID'] = out_df['Name'].apply(find_player_numbers, args = (players_df,))
                        out_df['Date'] = row.Date
                        out_df['Event'] = row.Event
                        out_df['Event_ID'] = row.Event_ID
                # End Pairs
                
                elif row.Source == 'TS': # Teams
                    n_teams = row.Tables
                    
                    teams_df = read_teams_results(csv_file, n_teams)
                    
                    out_df = ext_teams_winning_names(teams_df, row.Boards, row.Tables, row.Scale, single_winner=True)
    
                    if out_df is not None:
                        out_df['Name'] = out_df['Name'].fillna('')
        
                        out_df['Player ID'] = out_df['Name'].apply(find_player_numbers, args = (players_df,))
                        out_df['Date'] = row.Date
                        out_df['Event'] = row.Event
                        out_df['Event_ID'] = row.Event_ID
    
                # End Teams
                elif row.Source == 'SP':
                    print(event)
                elif row.Source == 'ST':
                    print(event)
            
            all_mp_df = all_mp_df.append(out_df)                     
            # End single item event
            
        elif len(event_df) > 1:
            print(event)
            
            # End multiple item event

Mixed Team Trials
Open Team Trials


In [69]:
unique_events

array(['New Year Pairs', 'Thursday Pairs', 'Friday Pairs',
       'Mixed Team Trials', 'Monday Pairs', 'NUS Open Pairs',
       'Open Team Trials'], dtype=object)

In [115]:
all_mp_df

,Position,Pair,Name,Cat,Adjust,Score,Tops,Percent,MPTS,Green,Player ID,Date,Event,Event_ID
0,1,1,Alan Sia,NaN,0,142.5,220,64.77,45.0,0.0,9050.0,2019-01-01,Tuesday Pairs New Year Day,200.0
8,2,9,Yoke Lan Foo,NaN,0,135.5,220,61.59,36.0,0.0,9017.0,2019-01-01,Tuesday Pairs New Year Day,200.0
7,3,8,Kheng Hun Siew,NaN,0,124.5,220,56.59,27.0,0.0,3557.0,2019-01-01,Tuesday Pairs New Year Day,200.0
2,4,3,Sida Wu,NaN,0,117.5,220,53.41,18.0,0.0,9082.0,2019-01-01,Tuesday Pairs New Year Day,200.0
11,5,13,Greta Chai,NaN,0,111.5,220,50.68,9.0,0.0,7023.0,2019-01-01,Tuesday Pairs New Year Day,200.0
0,1,1,Kimiko Endo,NaN,0,142.5,220,64.77,45.0,0.0,9095.0,2019-01-01,Tuesday Pairs New Year Day,200.0
8,2,9,Kenneth Chan,NaN,0,135.5,220,61.59,36.0,0.0,5139.0,2019-01-01,Tuesday Pairs New Year Day,200.0
7,3,8,Jun Yang Ang,NaN,0,124.5,220,56.59,27.0,0.0,9067.0,2019-01-01,Tuesday Pairs New Year Day,200.0
2,4,3,Wittmann Goh,NaN,0,117.5,220,53.41,18.0,0.0,9190.0,2019-01-01,Tuesday Pairs New Year Day,200.0
11,5,13,Trevor Li,NaN,0,111.5,220,50.68,9.0,0.0,1620.0,2019-01-01,Tuesday Pairs New Year Day,200.0


In [ ]:
def calc_match_mp(boards, in_df):
    
    

In [5]:
def calc_tables_phantom(df_meta):
    # Calculate Number of tables and phantom pair exists
    n_tables = int(df_meta.loc['#Tables'].value)
    n_boards = int(df_meta.loc['#Boards'].value)
    try:
        n_phantom = int(df_meta.loc['#Phantom'].value)
    except:
        n_phantom = 0
    if n_phantom > 0:
        n_phantom = 1
    return n_tables, n_boards, n_phantom

In [9]:
def find_player_numbers(name, table_df):
    name_words = name.split(' ')
    last_name = name_words[-1]
    first_name = ' '.join(name_words[:-1])
    
    try:
        player_id = int(players_df[(players_df['First Name']==first_name) & (players_df['Last Name']==last_name)]['ID'].item())
        return player_id
    except:
        return None
    

In [11]:
def read_from_csv(csv_file):
    # Read Metadata first - take first 20 rows
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20)
    
    n_tables, n_boards, n_phantom = calc_tables_phantom(df_meta)
    
    if int(df_meta.loc['#Winners'].value)!=1:
        single_winner=False
    else:
        single_winner=True
    
    max_pair = 2 * n_tables - n_phantom
    
    # Read actual results
    df = pd.read_csv(csv_file, sep=';',comment='#',header=1, nrows=50, error_bad_lines=False,warn_bad_lines=False)
    if 'Position' in df['Position'].values:
        df.drop(df[df['Position']=='Position'].index, inplace=True) # Clear up extra header for 2-winner
    pairs_df = df.iloc[0:int(max_pair)].sort_values('Position',ascending=True)
    
    if 'NS' in pairs_df['Position'].values:
        pairs_df.drop(pairs_df[pairs_df['Position']=='NS'].index, inplace=True) # Clear up extra header for missing phantom
        n_tables = n_tables - 1
        
    pairs_df['Position'] = pairs_df['Position'].astype('int') # Convert position to int
    
    out_df = ext_winning_names(pairs_df, n_boards, n_tables, 'Club', single_winner)
    
    if out_df is not None:
    
        out_df['Player ID'] = out_df['Name'].apply(find_player_numbers, args = (players_df,))
    
        out_df['Date'] = df_meta.loc['#Date'].value
        out_df['Event'] = df_meta.loc['#Title'].value
        out_df['Event_ID'] = df_meta.loc['Event Number'].value
    
        event_meta_df = ext_meta(df_meta)
    
        return out_df, event_meta_df
    else:
        event_meta_df = ext_meta(df_meta)
        return None, event_meta_df

In [258]:
pairs_mp_df = pd.DataFrame()
pairs_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"PS*.csv"):
    print(csv_file)
    out_df, event_meta_df = read_from_csv(csv_file)
    pairs_mp_df = pairs_mp_df.append(out_df,ignore_index=True)
    pairs_event_meta_df = pairs_event_meta_df.append(event_meta_df,ignore_index=True)

C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr01-2019-244.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr04-2019-245.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr05-2019-246.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr08-2019-247.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr11-2019-248.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr12-2019-249.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr15-2019-250.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr18-2019-1.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-14.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-2.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-5.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr22-2019-6.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr25-2019-7.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr26-2019-8.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr29-2019-9.csv
C:\misc\scba-techcomm\masterpoints\csv_2

# Swiss Pairs

# Teams

In [24]:
teams_csv_list = glob.glob(csv_dir+"TS*.csv")

In [18]:
def ext_teams_meta(df_meta):
    event_meta_df = pd.DataFrame()
    event_meta_df['Date'] = df_meta.loc['#Date']
    event_meta_df['Event'] = df_meta.loc['#Title']
    event_meta_df['Event_ID'] = df_meta.loc['Event Number']
    event_meta_df['Director'] = df_meta.loc['#Director']
    event_meta_df['Boards'] = int(df_meta.loc['#Boards'].value)
    event_meta_df['Tables'] = int(df_meta.loc['#Tables'].value)
    event_meta_df['Rounds'] = int(df_meta.loc['#Rounds'].value)
    event_meta_df.reset_index()
    
    return event_meta_df

In [21]:
teams_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"TS*.csv"):
    print(csv_file)
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20)
    event_meta_df = ext_teams_meta(df_meta)
    
    teams_event_meta_df = teams_event_meta_df.append(event_meta_df,ignore_index=True)

D:\scba-techcomm\masterpoints\csv_2019\TSApr06-2019-146.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr07-2019-149.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr21-2019-152.csv
D:\scba-techcomm\masterpoints\csv_2019\TSAug04-2019-174.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-185.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-187.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-188.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-189.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec15-2019-195.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan05-2019-120.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan06-2019-123.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan13-2019-128.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan19-2019-132.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan20-2019-134.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan26-2019-137.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan27-2019-140.csv
D:\scba-techcomm\masterpoints\csv_2019\T

In [33]:
out_df, meta_df = read_teams_from_csv(teams_csv_list[8])

In [51]:
out_df['Name'].replace('',np.nan, inplace=True)

In [52]:
out_df.dropna(how='all',subset=['Name','Player ID'],axis=0)

,Position,Team,Teamname,Name,Cat,Adjust,Imps,Vps,Matches,Matches2,Matches3,MPTS,Green,Player ID,Date,Event,Event_ID
1,2,1,Coup de Grace,Kelvin Ng,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,5017.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Timothy Wu,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,9186.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Linus Lee,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,9227.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Bryan Yang,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,9228.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Xue Heng Teo,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,5532.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Zhen Huai Toh,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9155.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Jie Min Kho,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9075.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Gideon Tan,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9209.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Jazlene Ong,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9261.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Edmund Chay,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9072.0,16/11/2019,H S Ho Qualifying,186


In [101]:
teams_mp_df = pd.DataFrame()
teams_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"TS*.csv"):
    print(csv_file)
    out_df, event_meta_df = read_teams_from_csv(csv_file)
    out_df.sort_values(by='Team',axis=0,inplace=True,kind='mergesort')
    out_df['Name'].replace('',np.nan, inplace=True)
    out_df.dropna(subset=['Name'],inplace=True)
    teams_mp_df = teams_mp_df.append(out_df,ignore_index=True)
    teams_event_meta_df = teams_event_meta_df.append(event_meta_df,ignore_index=True)

D:\scba-techcomm\masterpoints\csv_2019\TSApr06-2019-146.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr07-2019-149.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr21-2019-152.csv
D:\scba-techcomm\masterpoints\csv_2019\TSAug04-2019-174.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan05-2019-120.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan06-2019-123.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan13-2019-128.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan19-2019-132.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan20-2019-134.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan26-2019-137.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan27-2019-140.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJun05-2019-170.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay04-2019-158.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay11-2019-16.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay11-2019-161.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay12-2019-164.csv
D:\scba-techcomm\masterpoints\csv_2019\TS

In [103]:
teams_mp_df['Event'].value_counts()

Singapore Open Teams 2019 Qual    64
Singapore Open Team Consol RR2    43
National School Teams 2nd Day     39
Singapore Open Team Consol RR1    36
Mixed Teams 2019 RR2              31
Mixed Teams 2019 RR1              30
ABPF Open Team Trials Day 2       28
Mixed Team Trials Day 2           26
APBF Open Team Trials             26
Mixed Trials 2019                 24
H S Ho Qualifying                 20
Singapore Open Team Final 2019    13
ABPF Open Team Trials Day 4       11
Pesta Sukan Open Team Final       11
ABPF Open Team Trials              9
Mixed Team Trials Final 2019       8
EC Open Teams Finals               3
Name: Event, dtype: int64

In [102]:
teams_mp_df.to_csv('teams_mp_2019.csv',index=False)

In [84]:
filter_col = [col for col in teams_mp_df if col.startswith('Matches')]
filter_col_a = filter_col + ['Team']

In [85]:
filter_col

['Matches',
 'Matches10',
 'Matches2',
 'Matches3',
 'Matches4',
 'Matches5',
 'Matches6',
 'Matches7',
 'Matches8',
 'Matches9']

In [70]:
teams_mp_df[teams_mp_df['Event']=='Handicap Teams'][filter_col].groupby('Team').sum()

,Vps
Team,
1,443.40
2,287.92
3,182.94
4,21.78
5,132.04
6,79.93
7,433.54


In [86]:
tmp = teams_mp_df[teams_mp_df['Event']=='Handicap Teams'][filter_col_a]

In [77]:
def return_win(x):
    if x > 10:
        return 1
    else:
        return 0

In [90]:
tmp.loc[:, tmp.columns != 'Team'] = tmp[filter_col].applymap(return_win)

In [100]:
tmp[tmp['Team']==6]

,Matches,Matches10,Matches2,Matches3,Matches4,Matches5,Matches6,Matches7,Matches8,Matches9,Team
88,0,0,0,0,0,1,0,0,0,0,6
89,0,0,0,0,0,1,0,0,0,0,6
109,0,0,0,0,0,0,0,0,0,0,6
110,0,0,0,0,0,0,0,0,0,0,6
131,0,0,0,0,1,0,0,0,0,0,6
132,0,0,0,0,1,0,0,0,0,0,6
133,0,0,0,0,1,0,0,0,0,0,6
156,0,0,0,0,0,0,0,0,0,0,6
